# IEEE Fraud Detection

In [ ]:
# Reducing memory size by changing the data types.
# https://www.kaggle.com/mjbahmani/reducing-memory-size-for-ieee
# https://www.kaggle.com/arjanso/reducing-dataframe-memory-size-by-65

import os
import gc
gc.enable()
import time


In [ ]:
# %%time
# import Dataset to play with it
train_identity= pd.read_csv(os.getcwd() + '\\Data\\train_identity.csv', index_col='TransactionID')
train_transaction= pd.read_csv(os.getcwd() + '\\Data\\train_transaction.csv', index_col='TransactionID')
test_identity= pd.read_csv(os.getcwd() + '\\Data\\test_identity.csv', index_col='TransactionID')
test_transaction = pd.read_csv(os.getcwd() + '\\Data\\test_transaction.csv', index_col='TransactionID')
print ("Done!")

In [ ]:
print('Shape of Data:')
print(train_transaction.shape)
print(test_transaction.shape)
print(train_identity.shape)
print(test_identity.shape)

In [ ]:
# Create train & test dataset
train = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)
test = test_transaction.merge(test_identity, how='left', left_index=True, right_index=True)

del train_identity,train_transaction,test_identity, test_transaction

In [ ]:
train.info()

In [ ]:
test.info()

## IEEE Reducing Memory Size

In [ ]:
def reduce_mem_usage(df):
    start_mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in df.columns:
        if df[col].dtype != object:  # Exclude strings            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",df[col].dtype)            
            # make variables for Int, max and min
            IsInt = False
            mx = df[col].max()
            mn = df[col].min()
            print("min for this col: ",mn)
            print("max for this col: ",mx)
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(df[col]).all(): 
                NAlist.append(col)
                df[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = df[col].fillna(0).astype(np.int64)
            result = (df[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        df[col] = df[col].astype(np.uint8)
                    elif mx < 65535:
                        df[col] = df[col].astype(np.uint16)
                    elif mx < 4294967295:
                        df[col] = df[col].astype(np.uint32)
                    else:
                        df[col] = df[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)    
            # Make float datatypes 32 bit
            else:
                df[col] = df[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",df[col].dtype)
            print("******************************")
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return df, NAlist

In [ ]:
train, NAlist = reduce_mem_usage(train)
print("_________________")
print("")
print("Warning: the following columns have missing values filled with 'df['column_name'].min() -1': ")
print("_________________")
print("")
print(NAlist)

In [ ]:
test, NAlist = reduce_mem_usage(test)
print("_________________")
print("")
print("Warning: the following columns have missing values filled with 'df['column_name'].min() -1': ")
print("_________________")
print("")
print(NAlist)

In [ ]:
train.info()

In [ ]:
test.info()

## Saving reduced Dataset

In [ ]:
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

del train, test, NAlist

# Exploratory Data Analysis

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter("ignore")

In [2]:
train_data = pd.read_csv(os.getcwd() + '\\Data\\train.csv')
test_data = pd.read_csv(os.getcwd() + '\\Data\\test.csv')

In [3]:
train_data.info()
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Columns: 433 entries, isFraud to DeviceInfo
dtypes: float64(80), int64(322), object(31)
memory usage: 1.9+ GB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506691 entries, 0 to 506690
Columns: 432 entries, TransactionDT to DeviceInfo
dtypes: float64(80), int64(321), object(31)
memory usage: 1.6+ GB


In [24]:
print("The class proportion in Training Data:")
print("Yes : No =  {} : {}".format(round(sum(train_data['isFraud']==1)/train_data['isFraud'].shape[0],3),round(sum(train_data['isFraud']==0)/train_data['isFraud'].shape[0],3)))
print("Highly Imbalanced data set")


The class proportion in Training Data:
Yes : No =  0.035 : 0.965
Highly Imbalanced data set
